# Selection sort

In [1]:
:set -XNoMonadFailDesugaring
:l SortersM
:l Predicates

It turns out, that combining the default implementation of selection sort with `coinCmp` leads to enumerating multiple times some of the sequences - we get a supermultiset of the permutation set.

In [2]:
selectSortM coinCmp [3,2,1] :: [[Int]]

[[3,2,1],[3,1,2],[2,3,1],[2,1,3],[3,2,1],[3,1,2],[1,3,2],[1,2,3]]

The reason behind this, is the inconsistency of our predicate - during algorithm we may ask multiple times about the same pair of elements, but `coinCmp` may return different answers and thus leading to extra results. We can overcome this problem by guarding the consistency - we do that by keeping a *state* within our nondeterministic procedure, namely on every branch we keep track of the already built relation.

In [3]:
import Control.Monad.State
evalStateT (selectSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[1,3,2],[1,2,3]]

To avoid using monadic tranformers and states, we can instead use Set monad.

In [4]:
import qualified Data.Set.Monad as SetM (toList)
SetM.toList $ selectSortM coinCmp [3,2,1]

[[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]]

However, this approach is kind of inefficient. We sieve permutations after creating them. We may try to optimize this process one level lower - while seeking for minimum.

Here is the default implementation, but with inlined code of `coinCmp` and simplifications:

In [5]:
minimumNDM :: (MonadPlus m) => [a] -> m a
minimumNDM [x] = return x
minimumNDM (x:xs) = do
    y <- minimumNDM xs
    return x `mplus` return y

minimumNDM [1,2,3] :: [Int]

[1,2,1,3]

If we assume, that our monad is both idempotent wrt `mplus` and left distributive wrt `bind` and `mplus` we can simplify the implementation even more, avoiding calling `return x` unnecessary. As the list monad doesn't satisfy these requirements, we lose some of the results:

In [6]:
minimumNDM :: (MonadPlus m) => [a] -> m a
minimumNDM [x] = return x
minimumNDM (x:xs) = return x `mplus` minimumNDM xs

minimumNDM [1,2,3] :: [Int]

[1,2,3]

Fortunately, it is sufficient to get exactly `n!` unique permutations with inconsistent predicate (we have abandoned tracking answers).

In [7]:
import qualified Data.List as List

selectSortNDM :: (Eq a, MonadPlus m) => [a] -> m [a]
selectSortNDM [] = return []
selectSortNDM xs = do
    x <- minimumNDM xs
    fmap (x:) (selectSortNDM (List.delete x xs))

selectSortNDM [1,2,3] :: [[Int]]

[[1,2,3],[1,3,2],[2,1,3],[2,3,1],[3,1,2],[3,2,1]]